# Spectra Analysis Notebook - ORBYTS 2021/2022

$\textbf{Author:}$ Ryan Brady | $\textbf{Last version updated:}$ 20/12/2021

$\textbf{Contact Details:}$ If you have any questions outside of session, please ask them directly to Mr Cummings and he will forward them to me. I'll always try to answer them as quick as I can so you can crack on with the science! 

## About: 

Hi everyone, this is a Jupyter notebook for the analysis of high-resolution room-temperature spectra of Fromaldehyde - $H_2CO$.

## Aims & Objectives:

$\textbf{Aims:}$ 
- We aim to analyse the $H_2CO$ spectrum in un-assigned regions. This involves the assignment of spectral lines via $\textbf{quantum numbers}$ describing $Rotational$ (J) and $Vibrational$ (v) transitions within the molecule. 

- We will extract information on the broadening of lines - due to pressure, temperature, and composition of the gas mixture - and intensities (heights of the lines) - which tell us about temperature and other effects like whether the gas is in Local Thermodynamic equillibrium (LTE) or not.

$\textbf{Objectives:}$ 
- Compare the experimental spectra to synthetic spectra of $H_2CO$ for the assignment of quantum numbers.
- Extract broadening and intensities using the below Jupyter notebook (if you feel like you can improve the code, please feel free to have a go and put your changes in the notebook titled 'coding sandbox'!).

## Importing the required packages

Here we import packages containing functions we want to use in our code! Don't worry too much about this, you just need to know to run the below cell before anything else everytime you initialize the kernal!

In [1]:
## First we import the required modules and change their call name, e.g. instead of numpy we can write np
import numpy as np
import pandas as pd
import scipy
from scipy import integrate
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from matplotlib.widgets import SpanSelector

%matplotlib

Using matplotlib backend: MacOSX


## Defined Functions 
Here I will put all the defined functions that we want to use in the code. Modules and libraries dont have everything, so sometimes we define our own. Below includes the definition of our interactive plotter and the area under a lorentzian (like a bell shaped curve/normal distribution (Guassian) but with slower decaying wings).

In [2]:
## User selection function required for our interactive plotting and the computation of spectroscopic parameters
def lorentzian(r,w,h,r0):                                                       
    N = h/np.max((1/(2*w))*((w**2)/(((r - r0)**2) + (w**2))))
    
    return (N/(2*w))*((w**2)/(((r - r0)**2) + (w**2)))

def lor_area(r,w,h,r0): #area of a lorentzian
    return (h*np.pi)/(2*np.max((1/(2*w))*((w**2)/(((r - r0)**2) + (w**2)))))

def FWHM(x,y):  # FWHM = Full Width at Half Maximum
    ymax = np.max(y)
    HM = ymax/2
    
    x_trunc = []
    index = []
    
    for i in range(len(x)):
        if y[i]>=HM:
            x_trunc.append(x[i])
            index.append(i)
    
    return np.max(x_trunc)-np.min(x_trunc)



def header_and_data(df,header):
    return np.array(df[header]), header


def range_finder(n, minn, maxn):
    
    n_trunc = []
    index = []
    
    for i in range(len(n)):
        if minn<= n[i] <= maxn:
            n_trunc.append(n[i])    
            index.append(i)
            
    indexmin = min(index)  
    indexmax = max(index)
    
    return [n_trunc, indexmin, indexmax]

def n_max(n, indexmin, indexmax):
    return n[indexmin:indexmax].max()


def onselect(xmin, xmax):
    
    # find indices of the array elements corresponding to min and max energies in selection range.
    indmin, indmax = np.searchsorted(x, (xmin, xmax))
    indmax = min(len(x) - 1, indmax)
    
    # Re-define x, y variables to selected energies and intensities to plot
    selected_x = x[indmin:indmax]  # 'slices' the x (energy) array between indices indmin and indmax. 
    selected_y = y[indmin:indmax]  # "
    
    # Re-plot the selected data with axis limits. 
    line2.set_data(selected_x, selected_y)             # updates data in line2
    ax2.set_xlim(selected_x[0], selected_x[-1])        # set x axis limits to energy range in selection
    ax2.set_ylim(selected_y.min(), selected_y.max())   # set y axis limits to intensity range in selection
    fig.canvas.draw_idle()                             # updates the bottom plot.
    
    #
    I_max = n_max(I,range_finder(nu,selected_x[0], selected_x[-1])[1], range_finder(nu,selected_x[0], selected_x[-1])[2])
    
    ax3.set_xlim(selected_x[0], selected_x[-1])        # set x axis limits to energy range in selection
    ax3.set_ylim(0, I_max)                             # set y axis limits to intensity range in selection
    fig.canvas.draw_idle()          
    
    # Determine the area under the line profile, line height, and line position
    global L_height
    global L_pos
    global L_area
    global L_fit_popt
    global L_fit_area
    global u
    global o
    
    u = selected_x
    o = selected_y
    
    L_height = np.max(selected_y)                              # Find max line intensity        
    L_pos = selected_x[np.where(selected_y==L_height)]         # Find position of line centroid
    L_area = scipy.integrate.trapz(selected_y, selected_x)      # Find area under line == intensity with no broadening
    L_fit_popt, _ = curve_fit(lorentzian, selected_x, selected_y)
    L_fit_area = lor_area(selected_x, *L_fit_popt)
    
    
    
    # example of how to save selected data range, if you wanted.
    #np.savetxt("text.out", np.c_[selected_x, selected_y])

    
def onselect_QN(xmin, xmax):
    # find indices of the array elements corresponding to min and max energies in selection range.
    indmin, indmax = np.searchsorted(x, (xmin, xmax))
    indmax = min(len(x) - 1, indmax)
    
    # Re-define x, y variables to selected energies and intensities to plot
    selected_x = x[indmin:indmax]  # 'slices' the x (energy) array between indices indmin and indmax. 
    selected_y = y[indmin:indmax]  # "
    
    # Extrct QN of line
    index, i, j = range_finder(nu, selected_x[0], selected_x[-1])
    
    global QN
    QN = df_predicted.iloc[[i]]
    print(i)


## Importing Data and Plotting the Spectra

Here we import the experimental and synethetic data on the OH spectrum into dataframes. These are like a coding equivalent of a table in excel, where each column has a header. Here is a summary of the data used:

- 'OH_09032021_1_NBW_PZF2_S3M_1700-7900.csv' contains the experimental data for the reaction detailed above. The first column is the 'Energy' (wavenumbers, in $cm^{-1}$) of the emission lines in the spectra (x-axis) and each of the following columns are the intensites of the spectrum (arbitrary units) in steps of 2 ms. The headers for these are 'E' and 't1, t2,t3,...,t27' for each of the time series, repsectively.
- 

In [3]:
## Import data
df_exp = pd.read_csv("H2CO_exp_spec_6546-6900_cm-1.txt", delimiter=r"\s+")
x = np.array(df_exp['E'])
y = np.array(df_exp['I'])

df_predicted = pd.read_csv("H2CO_calc_dataframe.csv")
nu = np.array(df_predicted['nu'])
I  = np.array(df_predicted['I'])


## Plot data
fig = plt.figure(figsize=(8,6))

ax = fig.add_subplot(311)
ax.plot(x, y, '-', label = "Experimental spectrum")
ax.set_title('Press left mouse button and drag to zoom')
plt.legend()

ax2 = fig.add_subplot(312)
line2, = ax2.plot(x, y, '-')


ax3 = fig.add_subplot(313)
line3, = ax3.plot(x,np.zeros(len(x)))
line4 = ax3.stem(nu, I, markerfmt=' ', linefmt='magenta',label="model", use_line_collection = True)

ax3.set_yscale('log')

ax3.legend(loc='upper center', bbox_to_anchor=(0.5, -0.3),
          fancybox=True, shadow=True, ncol=5, fontsize=8.75)

fig.text(0.5, 0.055, r'Wavenumbers ($cm^{-1}$)', ha='center', va='center')
fig.text(0.04, 0.65, 'Intensity (arbitrary units)', ha='center', va='center', rotation='vertical', fontsize=10)
fig.text(0.02, 0.25, r'$Log_{10}$ Cross-section ', ha='center', va='center', rotation='vertical', fontsize=10)
fig.text(0.04, 0.25, r'($cm^2$ per molecule)', ha='center', va='center', rotation='vertical', fontsize=10)


## Below code allows the user to select areas of the spectra to zoom in on, and updates ax, ax2, and ax3. 
span = SpanSelector(ax, onselect, 'horizontal', useblit=True, rectprops=dict(alpha=0.5, facecolor='red'))
span2 = SpanSelector(ax2, onselect, 'horizontal', useblit=True, rectprops=dict(alpha=0.5, facecolor='red'))
span3 = SpanSelector(ax3, onselect_QN, 'horizontal', useblit=True, rectprops=dict(alpha=0.5, facecolor='blue'))

In [55]:
#print("max Intensity:", L_height)
#print("Position: ", L_pos)
#print("Area: ", L_area)
#print("fitted_area: ", L_fit_area)
#print("param: ", L_fit_popt)

QN.to_csv('Quantum_Numbers.txt')

a_file = open("Quantum_Numbers.txt")
lines = a_file. readlines()
for line in lines:
    print(line)


,nu,I,tau,Gamma',E',J',Ka',Kc',GammaRot',v1',v2',v3',v4',v5',v6',GammaVib',Unnamed: 16,"Gamma ""","E""","J""","GammaVib""","Ka""","Kc""","GammaRot""",Unnamed: 24

3217,6793.768026000001,2.4399999999999996e-23,0.0345,B1,6914.710016,10,1,9,B1,0,2,0,1,0,0,A1,<-,B2,120.94198999999999,9,A1,1,8,B2,

